In [ ]:
import requests
import os 
import json
import pandas as pd
import matplotlib.pyplot as plt
import time

In [ ]:
# Load apikey data

os.chdir('/Users/joemccann/MLProjects/alpaca_practice')
api_keys = json.load(open('apikey.json', 'r'))

In [ ]:
# Historical data functions
def get_bar_data(api_keys, url, symbols, start='', end='', limit=1000, timeframe='1Day'):
    list_of_dfs = []
    
    for symbol in symbols:
        headers = {
            "accept": "application/json",
            "APCA-API-KEY-ID": api_keys['APCA-API-KEY-ID'],
            "APCA-API-SECRET-KEY":  api_keys['APCA-API-SECRET-KEY']
        }
        
        params = {'symbols': symbol, 
              'timeframe':timeframe, 
              'start':start, 
              'end': end,
              'limit': limit}
    
        response = requests.get(url, headers=headers, params=params)
    
        data = response.json()
    
        df = pd.DataFrame(data['bars'][symbol])
        df.columns = ['close', 'high', 'low', 'trade_count', 'open', 'time', 'volume', 'volume_weighted_avg']
        df['time'] = pd.to_datetime(df['time'])
        df['stock_name'] = symbol
        df = df[['stock_name'] + [col for col in df.columns if col != 'stock_name']]
        df = df[['stock_name', 'time','open', 'close', 'high', 'low', 'trade_count', 'volume', 'volume_weighted_avg']]

        list_of_dfs.append(df)

    combined_df = pd.concat(list_of_dfs, ignore_index=True)

    return combined_df

def get_historical_quotes(api_keys, url, symbols, start='', end='', limit=1000):

    list_of_dfs = []

    for symbol in symbols:
        headers = {
            "accept": "application/json",
            "APCA-API-KEY-ID": api_keys['APCA-API-KEY-ID'],
            "APCA-API-SECRET-KEY":  api_keys['APCA-API-SECRET-KEY']
        }
        
        params = {'symbols': symbol, 
              'start':start, 
              'end': end,
              'limit': limit}
    
        response = requests.get(url, headers=headers, params=params)
    
        data = response.json()
    
        df = pd.DataFrame(data['quotes'][symbol])
        df.columns = ['ask_price', 'ask_size', 'ask_exchange', 'bid_price', 'bid_size', 'bid_exchange', 'condition_flag', 'timestamp', 'tape']
        df['stock_name'] = symbol
        df = df[['stock_name'] + [col for col in df.columns if col != 'stock_name']]

        list_of_dfs.append(df)

    combined_df = pd.concat(list_of_dfs, ignore_index=True)

    return combined_df

# Latest data functions
def get_lastest_bar(api_keys, url, stocks):
    headers = {
        "accept": "application/json",
        "APCA-API-KEY-ID": api_keys['APCA-API-KEY-ID'],
        "APCA-API-SECRET-KEY":  api_keys['APCA-API-SECRET-KEY']
    }

    list_of_dfs = []

    for stock in stocks:
        params = {'symbols': stock}
        
        response = requests.get(url, headers=headers, params=params)
        
        data = response.json()
    
        last_bar_df = pd.DataFrame(data['bars']).T
        last_bar_df.columns = ['close', 'high', 'low', 'trade_count', 'open', 'time', 'volume', 'volume_weighted_avg']
        last_bar_df['stock_name'] = stock
        last_bar_df = last_bar_df[['stock_name'] + [col for col in last_bar_df.columns if col != 'stock_name']]
        list_of_dfs.append(last_bar_df)
    
    combined_df = pd.concat(list_of_dfs, ignore_index=True)

    return combined_df

In [ ]:
# Get Bar Data

In [ ]:
url = "https://data.alpaca.markets/v2/stocks/bars?"
symbols = ['AMZN', 'TSLA', 'AAPL']
start = '2020-01-10'
end =  '2024-01-10'

get_bar_data(api_keys, url, symbols, start, end)

In [ ]:
#Last Trade

In [ ]:
last_url = "https://data.alpaca.markets/v2/stocks/bars/latest?"
stocks = ['AMZN', 'TSLA', 'AAPL']

get_lastest_bar(api_keys, last_url, stocks)

In [ ]:
# Historical Quotes

In [ ]:
url = "https://data.alpaca.markets/v2/stocks/quotes?"
# symbols = 'TSLA'
symbols = ['AMZN', 'TSLA', 'AAPL']
start = '2020-01-10'
end =  '2024-01-10'

get_historical_quotes(api_keys, url, symbols, start, end)